### Scrape Soccerway Wide Table data

A script that returns a season's wide table data - total; home; away - from soccerway.com.

Th

In [27]:
import bs4
import requests
import json
import pandas as pd
import re
import os

## Step 1: Get IDs for Season

In [28]:
# FUNCTIONS

# Get all of the page's code
def getHTML(url):
    
    # gets around 'Access denied'

    def extract_source(url):
        agent = {"User-Agent":"Mozilla/5.0"}
        source=requests.get(url, headers=agent).text
        return source

    def extract_data(source):
        soup=bs4.BeautifulSoup(source, 'lxml')
        names=soup.findAll('title')
        #for i in names:
             #print(i)
        return soup

    soup = extract_data(extract_source(url))

    title = soup.title.text
    
    return title, soup


# Get IDs from HTML
def getIDs(soup):

    #res = re.search(r"var block = new Block('page_competition_1_block_competition_tables_13', 'block_competition_tables', {\"season_id\":14007,\"round_id\":41485,\"outgroup\":false,\"competition_id\":9,\"new_design_callback\":true});", str(soup))

    str_soup = str(soup)
    wanted_line = "var block = new Block\('page_competition_1_block_competition_tables_(\d+)', 'block_competition_tables', \{\"season_id\":(\d+),\"round_id\":(\d+),\"outgroup\":false,\"competition_id\":(\d+),"
    
    numbers = re.findall(r"{}".format(wanted_line), str_soup)

    block_id, season_id, round_id, competition_id = None, None, None, None
    
    if numbers:    
        block_id, season_id, round_id, competition_id = numbers[0][0], numbers[0][1], numbers[0][2], numbers[0][3]

    return block_id, season_id, round_id, competition_id

In [29]:
country = 'germany'
league = 'bundesliga'
start_year = 2019

season = str(start_year)+str(start_year+1)
url = f'https://soccerway.com/national/{country}/{league}/{season}/regular-season'

title, soup = getHTML(url)

if 'Page Not Found' in title:
    season = str(i) + '-' + str(i+1)
    url = f'https://soccerway.com/national/{country}/{league}/{season}'
    title, soup = getHTML(url)
    season = season.replace('-','') 

block_id, season_id, round_id, competition_id = getIDs(soup)
    
print(block_id, season_id, round_id, competition_id)

13 17512 53499 9


## Step 2: With IDs inserted into URL, get HTML of wide table data

In [30]:
url = f'https://int.soccerway.com/a/block_competition_tables?block_id=page_competition_1_block_competition_tables_{block_id}&callback_params=%7B"season_id"%3A{season_id}%2C"round_id"%3A{round_id}%2C"outgroup"%3Afalse%2C"competition_id"%3A{competition_id}%2C"new_design_callback"%3Atrue%7D&action=changeTable&params=%7B"type"%3A"competition_wide_table"%7D'

In [31]:
## Code to remove from scraped code
a = '<html><body><p>{"commands":[{"name":"updateContainer","parameters":{"content":"</p>'
b = ',"timestamp":"Sun, 22 Aug 2021 10:39:52 +0200"}</script></div></div></div></div></div></div></div></div></div></div></div></div></td></tr></tfoot></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></acronym></th></tr></thead></table></form></div></body></html>'

In [32]:
# gets around 'Access denied'

def extract_source(url):
    agent = {"User-Agent":"Mozilla/5.0"}
    source=requests.get(url, headers=agent).text
    return source

def extract_data(source):
    
    soup=bs4.BeautifulSoup(source, 'lxml')
    names=soup.findAll('title')
    for i in names:
         print(i)
        
    return soup

soup = extract_data(extract_source(url))

str(soup)[0:3000]

'<html><body><p>{"commands":[{"name":"updateContainer","parameters":{"content":"</p><div block="" block-nomb="" class=\'\\"block_competition_wide_table\' clearfix="" id=\'\\"page_competition_1_block_competition_tables_13_block_competition_wide_table_1\\"\'><form accept-charset=\'\\"utf-8\\"\' action=\'\\"\\/teams\\/comparison\\/\\"\' method=\'\\"get\\"\'><table class=\'\\"leaguetable\' data-round_id=\'\\"53499\\"\' detailed-table="" fixed-wide-table="" id=\'\\"page_competition_1_block_competition_tables_13_block_competition_wide_table_1_table\\"\' sortable="" table=""><thead><tr class=\'\\"main-head\\"\'><th class=\'\\"rank\\"\'>\xa0</th><th class=\'\\"direction\\"\'>\xa0</th><th class=\'\\"team\\"\'>\xa0</th><th class=\'\\"type\\"\' colspan=\'\\"6\\"\'>Total</th><th class=\'\\"type\\"\' colspan=\'\\"6\\"\'>Home</th><th class=\'\\"type\\"\' colspan=\'\\"6\\"\'>Away</th><th class=\'\\"number\' gd="">\xa0</th><th class=\'\\"number\' points="">\xa0</th></tr><tr class=\'\\"sub-head\\"\'><t

## Step 3: Extract the table data from HTML

In [34]:
df = pd.DataFrame()

table = soup.select('tbody')

for t in table:
    for tr in t.find_all('tr'):
        #for th in tr.find_all('th'):
        #print(tr)
        
        for td in tr.find_all('td'):  
            #print(td)
            number = td.text
            
            if 'team' in str(td):
                
                team = td.text
                        
            if 'number' in str(td):
                td = str(td).replace('\\\"number','') 
                td = td.replace("class=''",'')
                
                td = td.replace("<td",'')
                td = td.replace(f'>{number}</td>','')
                td = td.replace('=""','')
                
                td = td.strip()
                
                if '_' in td:                    
                    td = [x for x in td.split(' ') if '_' in x][0]
        
                    if '_total' in td:        
                        td = td.replace('_total','_mp')
                                                              
                else:
                    td = td.split(' ')
                    
                    if len(td) > 1:
                        if td[1] == 'total':
                            td = td[1] + '_' + td[0]   
                    else:
                        td = 'total_' + td[0]

                df.loc[team, td] = number


                
directory = f'CSVs/{country}/{league}/{season}/'
if not os.path.exists(directory):
    os.makedirs(directory)             
df.to_csv(f'{directory}/widetable.csv')             
                
df

,total_mp,total_won,total_drawn,total_lost,total_gf,total_ga,home_mp,home_won,home_drawn,home_lost,home_gf,home_ga,away_mp,away_won,away_drawn,away_lost,away_gf,away_ga,total_gd,total_points
Bayern Munich,34,26,4,4,100,32,17,13,2,2,53,15,17,13,2,2,47,17,+68,82
Borussia Do…,34,21,6,7,84,41,17,11,3,3,46,17,17,10,3,4,38,24,+43,69
RB Leipzig,34,18,12,4,81,37,17,7,8,2,38,21,17,11,4,2,43,16,+44,66
Borussia M'…,34,20,5,9,66,40,17,12,2,3,40,21,17,8,3,6,26,19,+26,65
Bayer Lever…,34,19,6,9,61,44,17,9,4,4,32,22,17,10,2,5,29,22,+17,63
Hoffenheim,34,15,7,12,53,53,17,7,1,9,26,37,17,8,6,3,27,16,+0,52
Wolfsburg,34,13,10,11,48,46,17,4,7,6,19,23,17,9,3,5,29,23,+2,49
Freiburg,34,13,9,12,48,47,17,9,2,6,23,17,17,4,7,6,25,30,+1,48
Eintracht F…,34,13,6,15,59,60,17,8,4,5,36,27,17,5,2,10,23,33,-1,45
Hertha Berlin,34,11,8,15,48,59,17,6,3,8,23,32,17,5,5,7,25,27,-11,41
